# Network Analysis

In [ ]:
import sys
import numpy as np
import tabulate
import time
import copy

import logging

from sklearn.datasets import make_sparse_spd_matrix
import matplotlib.pyplot as plt
import os
import sys
import re
import glob
import ast
import os.path as op
import pickle
import scipy
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp

from collections import defaultdict

from copy import deepcopy
import copy

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FastICA
from sklearn.utils import resample
from sklearn.covariance import GraphicalLassoCV
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression

from itertools import combinations
from scipy.stats import ttest_ind

import scipy.stats as stats

import networkx as nx
import glob, os
from functools import partial

from scipy.stats import pearsonr

import pickle

import warnings
warnings.filterwarnings("ignore")

## Load data

In [ ]:
dataset = 'train'
test = False if dataset == 'train' else False

Loading training data

In [ ]:
results_ern_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_models_{dataset}_id_clean.pkl")
results_ern_lat_demo_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_cov_fal_models_{dataset}_id_clean.pkl")
results_crn_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_models_{dataset}_id_clean.pkl")
results_crn_lat_demo_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_cov_fal2_models_{dataset}_id_clean.pkl")

results_ern_crn_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_crn_models_{dataset}_id_clean.pkl")
results_ern_crn_lat_demo_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_crn_cov_fal_models_{dataset}_id_clean.pkl")

train_datasets = [results_ern_df, results_ern_lat_demo_df, results_crn_df, results_crn_lat_demo_df, results_ern_crn_df, results_ern_crn_lat_demo_df]

Loading testing data

In [ ]:
results_ern_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_models_test_id_clean.pkl")
results_ern_lat_demo_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_cov_fal_models_test_id_clean.pkl")
results_crn_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_models_test_id_clean.pkl")
results_crn_lat_demo_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_cov_fal2_models_test_id_clean.pkl")

results_ern_crn_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_crn_models_test_id_clean.pkl")
results_ern_crn_lat_demo_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_crn_cov_fal_models_test_id_clean.pkl")

# fill nan
results_crn_lat_demo_test_df['e_LT_F2_C'] = results_crn_lat_demo_test_df['e_LT_F2_C'].fillna(results_crn_lat_demo_test_df['e_LT_F2_C'].mean())
results_ern_crn_lat_demo_test_df['e_LT_F2_C'] = results_ern_crn_lat_demo_test_df['e_LT_F2_C'].fillna(results_ern_crn_lat_demo_test_df['e_LT_F2_C'].mean())


test_datasets = [results_ern_test_df, results_ern_lat_demo_test_df, results_crn_test_df, results_crn_lat_demo_test_df, results_ern_crn_test_df, results_ern_crn_lat_demo_test_df]

Define names mapping and order

In [ ]:
mapping = {
    'e_ERN': "ERN",
    'e_CRN': "CRN",
    "RRQ": "RRQ",
    "DASS-21 Stress": "STR",
    "DASS-21 Dep": "DEP",
    "STAI-T": "ANX",
    "STAI-S Diff": 'AFL',
    "BIS": "BIS",
    "OBSESS": "OBS",
    "HOARD": "HRD",
    "ORD": "ORD",
    "CHECK": "CHK",
    "WBSI": "TSU",
    "IUS-P": "PIU",
    "IUS-I": "IIU",
    "SES": "SES",
    'BAS_D': "DRV",
    'BAS_PRZY': "FUN",
    'BAS_NAG': "RER",
    'INDEC_F': "IND",
    'PUN': "PUN",
    'REW': "REW",
    'HARM': "HRM",
    'T-CTR': "TCR",
    "OT": "THR",
    'OB_PERF': "PER",
    'PS': "STN",
    'G_SE': "GLT",
    'AMB': "AMB",
    'PRED': "NPR",
    'STAND': "STD",   
    "Age": "AGE",
    "Handness": "HND",
    'performance': "PRF",
    'e_LT_F': "LT-E",
    'e_LT_F2_C': "LT-C",
}

mapping_order = {value: index for index, value in enumerate(mapping.values())}
print(mapping_order)

Prepare datasets

In [ ]:
columns_to_drop_covs = ['WASH', 'NEU', 'Sex', 'id']
columns_to_drop = ['WASH', 'NEU',  'id']

# train dataset
ern_data_df_ = results_ern_df.drop(columns=columns_to_drop).rename(columns=mapping)
ern_cov_fal_data_df_ = results_ern_lat_demo_df.drop(columns=columns_to_drop_covs).rename(columns=mapping)

crn_data_df_ = results_crn_df.drop(columns=columns_to_drop).rename(columns=mapping)
crn_cov_fal2_data_df_ = results_crn_lat_demo_df.drop(columns=columns_to_drop_covs).rename(columns=mapping)

ern_crn_data_df_ = results_ern_crn_df.drop(columns=columns_to_drop).rename(columns=mapping)
ern_crn_cov_fal_data_df_ = results_ern_crn_lat_demo_df.drop(columns=columns_to_drop_covs).rename(columns=mapping)

# test dataset
ern_data_df_test_ = results_ern_test_df.drop(columns=columns_to_drop).rename(columns=mapping)
ern_cov_fal_data_df_test_ = results_ern_lat_demo_test_df.drop(columns=columns_to_drop_covs).rename(columns=mapping)

crn_data_df_test_ = results_crn_test_df.drop(columns=columns_to_drop).rename(columns=mapping)
crn_cov_fal2_data_df_test_ = results_crn_lat_demo_test_df.drop(columns=columns_to_drop_covs).rename(columns=mapping)

ern_crn_data_df_test_ = results_ern_crn_test_df.drop(columns=columns_to_drop).rename(columns=mapping)
ern_crn_cov_fal_data_df_test_ = results_ern_crn_lat_demo_test_df.drop(columns=columns_to_drop_covs).rename(columns=mapping)


datasets_train = [ern_data_df_, ern_cov_fal_data_df_, crn_data_df_, crn_cov_fal2_data_df_, ern_crn_data_df_, ern_crn_cov_fal_data_df_]
datasets_test = [ern_data_df_test_, ern_cov_fal_data_df_test_, crn_data_df_test_, crn_cov_fal2_data_df_test_, ern_crn_data_df_test_, ern_crn_cov_fal_data_df_test_]

### Read estimated graphs

In [ ]:
dir_ = f"../new_results/graphs/{dataset}/"

graphs = []

for file in sorted(glob.glob(dir_ + "*.gml")):
    print(file)
    graph = nx.read_gml(file)
    graphs.append(graph)
    
graphs

## Network analysis

### Define custom network measures 

In [ ]:
def calculate_nodes_predictability(X, G):
    explained_variance = dict()
        
    for node in G.nodes():
        y_ = X[[node]]

        neighbors = list(G.neighbors(node))

        X_ = X.loc[:, neighbors]

        lm = LinearRegression()
        lm.fit(X_, y_)

        score = lm.score(X_,y_)
        explained_variance[node] = score

    return explained_variance

In [ ]:
def calculate_nodes_predictability_train(X, G):
    explained_variance = dict()
        
    for node in G.nodes():
        y_ = X[[node]]

        neighbors = list(G.neighbors(node))

        X_ = X.loc[:, neighbors]

        lm = LinearRegression()
        lm.fit(X_, y_)

        score = lm.score(X_,y_)
        explained_variance[node] = score

    return explained_variance

In [ ]:
def current_flow_closeness_centrality(G, weight):
    G_ = copy.deepcopy(G.copy())
    for u, v, data in G_.edges(data=True):
        data['weight'] = abs(data['weight'])
        
    measure = nx.current_flow_closeness_centrality(G_, weight=weight)
    return measure

### Perform network analysis

In [ ]:
network_analysis_results_df = pd.DataFrame()

for i in range(0,len(graphs)):
    G = copy.deepcopy(graphs[i])
    dataset_train = datasets_train[i]
    dataset_test = datasets_test[i]

    measures = [
            (calculate_nodes_predictability, {'X': dataset_test}), 
            (calculate_nodes_predictability_train, {'X': dataset_train}), 
            (nx.degree_centrality, {}),
            (nx.closeness_centrality, {}),
            (current_flow_closeness_centrality, {'weight': 'weight'}),
            (nx.betweenness_centrality, {'weight': 'weight'}),
            (nx.current_flow_betweenness_centrality, {'weight': 'weight'}),
            (nx.load_centrality, {})
    ]
    
    for measure, measure_parameters in measures:
        G = copy.deepcopy(graphs[i])
        network_measure = measure(G = G, **measure_parameters)
        network_measure = {k: v for k, v in sorted(network_measure.items(), key=lambda item: item[0], reverse=True)}
        network_measure_nodes = network_measure.keys()
        network_measure_values = network_measure.values()
        network_measure_df = pd.DataFrame({
            'node': network_measure_nodes,
            'value' : network_measure_values,
            'graph' : [i] * len(network_measure_nodes),
            'measure' : [measure.__name__] * len(network_measure_nodes),
        })
        network_analysis_results_df = pd.concat([network_analysis_results_df, network_measure_df], ignore_index = True)

In [ ]:
# with pd.option_context('display.max_colwidth', None, 'display.max_columns', None,  'display.max_rows', None,):
#     display(network_analysis_results_df.groupby(['graph', 'measure']).apply(lambda a: a.drop(['graph', 'measure'], axis=1)[:]))

Save the results

In [ ]:
network_analysis_results_df.to_pickle('../new_results/network_analysis/base_estimates/network_analysis_results_with_diff_graphs.pkl')
network_analysis_results_df.to_csv('../new_results/network_analysis/base_estimates/network_analysis_results_with_diff_graphs.csv')

Save the results in pleasant form

In [ ]:
network_analysis_results_df = pd.read_pickle('../new_results/network_analysis/base_estimates/network_analysis_results_with_diff_graphs.pkl')
network_analysis_results_df.head()

In [ ]:
df = network_analysis_results_df.copy()
df = df[df['measure'].isin(['calculate_nodes_predictability', 'calculate_nodes_predictability_train', 'degree_centrality', 'current_flow_closeness_centrality'])]

df['node'] = df['node'].replace(mapping)
df = df.sort_values(by=['node'], key=lambda x: x.map(mapping_order))
df = df.pivot_table(index='node', columns=['graph', 'measure'])
df = df.sort_values(by=['node'], key=lambda x: x.map(mapping_order))

# reorder and rename network measures
net_colums_names_mapper = {
    'calculate_nodes_predictability' : 'P',
    'calculate_nodes_predictability_train' : 'P-W',
    'current_flow_closeness_centrality': 'C-F-C-C',
    'degree_centrality' : 'D-C',    
}

df = df.rename(columns=net_colums_names_mapper)

net_colums_names_order = ['P', 'P-W', 'D-C','C-F-C-C']
new_cols = df.columns.reindex(net_colums_names_order, level=2)

df = df.reindex(columns=new_cols[0])

with pd.option_context(
    'display.max_colwidth', None, 
    'display.max_columns', None,  
    'display.max_rows', None, 
    'display.float_format', '{:.3f}'.format):
    display(df)
    
df.to_csv('../new_results/network_analysis/base_estimates/network_analysis_results_with_diff_graphs_easy_view.csv')

### Visualize the results

In [ ]:
cm = 1/2.54
dpi = 500

sns.set_style("whitegrid")

plt.rcParams['figure.dpi'] = dpi
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['axes.labelsize'] = 6
plt.rcParams['axes.titlesize'] = 6
plt.rcParams["axes.edgecolor"] = ".15"
plt.rcParams["axes.linewidth"]  = 0.6
plt.rcParams['lines.linewidth'] = 0.6

for i in range(0,6):
    plt.figure(i)

    df = network_analysis_results_df.copy()
    df = df[df['measure'].isin(['calculate_nodes_predictability', 'degree_centrality', 'current_flow_closeness_centrality'])]
    df.node = df.node.astype(str)
    df_ = df[df['graph'] == i]

    df_['node'] = df_['node'].replace(mapping)
    df_ = df_.sort_values(by=['node'], key=lambda x: x.map(mapping_order))

    
    g = sns.FacetGrid(
        df_, 
        col='measure',
        col_order=['calculate_nodes_predictability', 'degree_centrality', 'current_flow_closeness_centrality'],
        sharex=False,
        height=8*cm,
        aspect=0.3,
        despine=False,
        gridspec_kws={"wspace":0.2}
    )

    g.map(
        sns.pointplot, 
        'value',
        'node',
        errorbar=None,
        color='black',
    )

    axes = g.axes.flatten()
    axes[0].set_title("Predictability")
    axes[1].set_title("D-Centrality")
    axes[2].set_title("C-F Closeness")
    
    g.set_axis_labels(x_var="")

    fig = g.fig
    fig.savefig(f'../new_results/network_analysis/base_estimates/network_analysis_dpi_{dpi}_graph_{i}_abs.png', bbox_inches='tight')
    
plt.show()

## MST

In [ ]:
def maximum_spanning_tree(G, edges_to_drop = None):
    G_ = G.copy()
    if edges_to_drop is not None:
        for k,v in edges_to_drop:
            if G_.has_edge(k, v):
                G_.remove_edge(k, v)
            else:
                print(f"The edge ({k}, {v}) does not exist in the graph.")
    # abs of weights
    # for u, v, data in G_.edges(data=True):
    #     data['weight'] = abs(data['weight'])
    
    tree_ = nx.maximum_spanning_tree(G_, 'prim')
    
    return (G_, tree_)  

In [ ]:
ern_cov_graph = graphs[1]
ern_crv_cov_graph = graphs[5]

In [ ]:
G_ = ern_crv_cov_graph
G_, T_ = maximum_spanning_tree(G_)

In [ ]:
T_.edges()

In [ ]:
cm = 1/2.54
dpi = 500

plt.rcParams['figure.dpi'] = dpi
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['ytick.labelsize'] = 7
plt.rcParams['xtick.labelsize'] = 7
plt.rcParams['axes.labelsize'] = 9
plt.rcParams["axes.edgecolor"] = ".15"
plt.rcParams["axes.linewidth"]  = 0.5
sns.set_style("white")
palette = sns.color_palette("colorblind")

fig = plt.figure(3, figsize=(10*cm, 10*cm))
axes = fig.add_axes([0,0,1,1], aspect=1)
axes.set_xlim([-1,1.3])
axes.set_ylim([-1.2,1.2])

    

# Find the maximum spanning tree
# T = nx.maximum_spanning_tree(G_)

# Visualize the graph and the minimum spanning tree
pos = nx.spring_layout(G_, scale=1, seed=1)
nx.draw_networkx_nodes(G_, pos, node_color="lightblue", node_size=50)
# nx.draw_networkx_edges(G_, pos, edge_color="grey")
nx.draw_networkx_labels(G_, pos, font_size=5, font_family="sans-serif")
nx.draw_networkx_edge_labels(
    G_, pos, edge_labels={(u, v): d["weight"] for u, v, d in G_.edges(data=True)}
)
nx.draw_networkx_edges(T_, pos, edge_color="green", width=1)
plt.axis("off")
plt.show()

In [ ]:
dir_graphs = '../new_results/graphs'
nx.write_gml(T_, os.path.join(dir_graphs, 'ern_crn_mst_weights.gml'))

In [ ]:
for u, v, d in T_.edges(data=True):
    print(f'{(u, v)}, {d["weight"]}') 

In [ ]:
def dijkstra_shortest_path(G, source, edges_to_drop = None):
    G_ = G.copy()
    if edges_to_drop is not None:
        for k,v in edges_to_drop:
            if G_.has_edge(k, v):
                G_.remove_edge(k, v)
            else:
                print(f"The edge ({k}, {v}) does not exist in the graph.")
    # abs of weights
    for u, v, data in G_.edges(data=True):
        data['weight'] = abs(data['weight'])
    
    # reverse of weights to make the highest weight the best
    for u, v, data in G_.edges(data=True):
        data['weight'] = 1/data['weight']
    
    path_ = nx.single_source_dijkstra_path(G_, source)
    
    return path_  

In [ ]:
def simple_shortest_path(G, source, edges_to_drop = None):
    G_ = G.copy()
    if edges_to_drop is not None:
        for k,v in edges_to_drop:
            if G_.has_edge(k, v):
                G_.remove_edge(k, v)
            else:
                print(f"The edge ({k}, {v}) does not exist in the graph.")
    
    path_ = nx.single_source_shortest_path(G_, source)
    
    return path_  

**ERN**

In [ ]:
ern_cov_graph = graphs[2]
ern_crv_cov_graph = graphs[5]

In [ ]:
ern_cov_graph.nodes()

In [ ]:
for u, v, d in ern_crv_cov_graph.edges(data=True):
    print(f'{(u, v)}, {d["weight"]}') 

- weighed

In [ ]:
G_ = ern_cov_graph
path = dijkstra_shortest_path(G_, 'PER')
path

In [ ]:
G_ = ern_crv_cov_graph
path = dijkstra_shortest_path(G_, 'PER')
path

- non weighed

In [ ]:
path = simple_shortest_path(G_, 'ANX')
path